In [1]:
import torch
import torch as t
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
from sklearn.feature_extraction.text import TfidfVectorizer
import os

In [12]:
print(torch.cuda.is_available())

False


In [13]:
import pandas as pd
import numpy as np
df = pd.read_csv('./memotion_dataset_7k/labels.csv')
df = df.iloc[:,1:]
df = df.fillna('')
image_names = df['image_name'].tolist()
extensions =[]
for i in image_names:
    extensions.append(i.split('.')[-1].lower())
print(set(extensions))

{'jpeg', 'png', 'jpg', 'bmp', 'jpe'}


In [14]:

#replacing Nan values
text = df['text_corrected'].to_numpy()
# average = " "
# text[np.isnan(text)] = average
humour = df['humour'].to_numpy()
# a,b = np.unique(humour,return_counts=True)
# average = a[np.argmax(b)]
# humour[np.isnan(humour)]=average

sarcasm = df['sarcasm'].to_numpy()
# a,b = np.unique(sarcasm,return_counts=True)
# average = a[np.argmax(b)]
# sarcasm[np.isnan(sarcasm)] = average

offensive = df['offensive'].to_numpy()
# a,b = np.unique(offensive,return_counts=True)
# average = a[np.argmax(b)]
# offensive[np.isnan(offensive)] = average

motivational = df['motivational'].to_numpy()
# a,b = np.unique(motivational,return_counts=True)
# average = a[np.argmax(b)]
# motivational[np.isnan(motivational)] = average

overall_sentiment = df['overall_sentiment'].to_numpy()
# a,b = np.unique(overall_sentiment,return_counts=True)
# average = a[np.argmax(b)]
# overall_sentiment = [average if np.isnan(x) else x for x in overall_sentiment]

In [15]:
set(humour)

{'funny', 'hilarious', 'not_funny', 'very_funny'}

# Applying TF_IDF vectorizer

In [16]:
#text = 'LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIKUT TREND PLAY THE 10 YEARS CHALLENGE AT FACEBOOK imgflip.com'
tfidf = TfidfVectorizer(stop_words='english')
text_tfidf = tfidf.fit_transform(text)

# Convert the tf-idf matrix to a PyTorch tensor
text_tensor = torch.Tensor(text_tfidf.toarray())

len(text_tensor)
len(text_tensor[0])

12657

In [17]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
humour = le.fit_transform(humour)
sarcasm = le.fit_transform(sarcasm)
offensive = le.fit_transform(offensive) 
motivational = le.fit_transform(motivational)
overall_sentiment = le.fit_transform(overall_sentiment)

In [18]:
a = torch.tensor([1,2,3])
a.float()

tensor([1., 2., 3.])

In [19]:
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, image_dir, text_list,humour, sarcasm,offensive,motivational,overall_sentiment, transform=None):
        self.image_dir = image_dir
        self.text_list = text_list
        self.humour = humour
        self.sarcasm = sarcasm
        self.offensive = offensive
        self.motivational = motivational
        self.overall_sentiment = overall_sentiment
        self.transform = transform

    def __len__(self):
        return len(self.text_list)

    def __getitem__(self, idx):
        # load image and convert to RGB
        try:
            img_path = os.path.join(self.image_dir, "image_"+str(idx)+".jpg")
            image = Image.open(img_path).convert('RGB')

        except:
            try:
                img_path = os.path.join(self.image_dir, "image_"+str(idx)+".png")
                image = Image.open(img_path).convert('RGB')


            except:
                try:
                    img_path = os.path.join(self.image_dir, "image_"+str(idx)+".jpeg")
                    image = Image.open(img_path).convert('RGB')
                except:
                    try:
                        img_path = os.path.join(self.image_dir, "image_"+str(idx)+".bmp")
                        image = Image.open(img_path).convert('RGB')
                    except:
                        img_path = os.path.join(self.image_dir, "image_"+str(idx)+".jpe")
                        image = Image.open(img_path).convert('RGB')


        #transform = transforms.Compose([transforms.PILToTensor()])
        transform = transforms.Compose([transforms.Resize((100,100)), transforms.ToTensor()])


        # transform = transforms.PILToTensor()
        # Convert the PIL image to Torch tensor
        
        image = transform(image)
        image = torch.tensor(image)
        image = torch.flatten(image)

        text = self.text_list[idx]
        humour = self.humour[idx]
        sarcasm = self.sarcasm[idx]
        offensive = self.offensive[idx]
        motivational = self.motivational[idx]
        overall_sentiment = self.overall_sentiment[idx]

        # return the sample as a dictionary
        sample = {'image': image, 'text': text,'humour':humour, 'sarcasm': sarcasm, 'offensive':offensive, 'motivational':motivational , 'overall_sentiment':overall_sentiment}
        return sample

# Example usage
#image_dir = "./memotion_dataset_7k/images"
image_dir = 'C:\\Users\\A1D\\Desktop\\MEME CLASSIFIER (MULTIMODAL)\\memotion_dataset_7k\\images'

dataset = MyDataset(image_dir, text_tensor, humour,sarcasm,offensive,motivational,overall_sentiment, transform=None)
dataloader = DataLoader(dataset, batch_size=5, shuffle=True)


In [20]:
import torch.nn as nn
import torch.nn.functional as F

class MemeClassifier(nn.Module):
    def __init__(self):
        super(MemeClassifier, self).__init__()
        #self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        #self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        #self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        
#         self.conv1 = nn.Conv1d(3000,5,kernel_size=3, padding=1)
#         self.conv2 = nn.Conv1d(32, 64, 3, padding=1)
#         self.conv3 = nn.Conv1d(64, 128, 3, padding=1)
        
        self.conv1 = nn.Conv1d(5, 30000, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(30000, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(128, 5, kernel_size=3, padding=1)         
        
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 16 * 16, 512)
        self.fc2 = nn.Linear(512, 256)

        #self.text_emb = nn.Embedding(num_embeddings=10000, embedding_dim=256)
        self.text_conv1 = nn.Conv1d(5, 12657, kernel_size=3, padding=1)
        self.text_conv2 = nn.Conv1d(12657, 1600, kernel_size=3, padding=1)
        self.text_pool = nn.MaxPool1d(2, stride=2)
        self.text_fc1 = nn.Linear(1600, 256)
        self.text_fc2 = nn.Linear(256, 5)

        self.fc3 = nn.Linear(6914, 20)
        self.fc4 = nn.Linear(20, 5)

    def forward(self, image, text):
        x1 = self.conv1(image)
        x1 = F.relu(x1)
        x1 = self.pool(x1)

        x1 = self.conv2(x1)
        x1 = F.relu(x1)
        x1 = self.pool(x1)

        x1 = self.conv3(x1)
        x1 = F.relu(x1)
        x1 = self.pool(x1)

        #x1 = x1.view(-1, 128 * 16 * 16)
        #x1 = F.relu(self.fc1(x1))
        #x1 = F.relu(self.fc2(x1))

        #text = self.text_emb(text)
        #text = text.permute(0, 2, 1)

        text = self.text_conv1(text)
        text = F.relu(text)
        text = self.text_pool(text)

        text = self.text_conv2(text)
        text = F.relu(text)
        text = self.text_pool(text)

        #text = text.view(-1, 64 * 25)
        text = F.relu(self.text_fc1(text.T))
        text = F.relu(self.text_fc2(text))
        print("x1:",np.shape(x1)," text: ", np.shape(text.T))
        x2 = torch.cat((x1, text.T), dim=1)
        x2 = F.relu(self.fc3(x2))
        x2 = self.fc4(x2)

        output_humor = x2[:, 0]
        output_sarcasm = x2[:, 1]
        output_offensive = x2[:, 2]
        output_motivational = x2[:, 3]
        output_overall_sentiment = x2[:,4]
        
        return output_humor, output_sarcasm, output_offensive, output_motivational, output_overall_sentiment


In [ ]:
import torch.optim as optim

model = MemeClassifier()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
num_epochs = 5

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        inputs, humour, sarcasm, offensive, motivational, overall_sentiment = data['image'],data['humour'], data['sarcasm'], data['offensive'], data['motivational'], data['overall_sentiment']
        print("batch of 50 images being processed...s")
        humour = humour.float()
        sarcasm = sarcasm.float()
        offensive = offensive.float()
        motivational = motivational.float()
        overall_sentiment = overall_sentiment.float()


        text = data['text']
        optimizer.zero_grad()

        # Forward pass
        output_humor, output_sarcasm, output_offensive, output_motivational, output_overall_sentiment = model(inputs, text)
        loss_humor = criterion(output_humor, humour)
        loss_sarcasm = criterion(output_sarcasm, sarcasm)
        loss_offensive = criterion(output_offensive, offensive)
        loss_motivational = criterion(output_motivational, motivational)
        loss_overall_sentiment = criterion(output_overall_sentiment, overall_sentiment)
        loss = loss_humor + loss_sarcasm + loss_offensive + loss_motivational

        # Backward pass
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        
        print("Batch discarded")
        
        if i % 10 == 9:    # Print every 10 mini-batches
            print('[Epoch: %d, Batch: %5d] Loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0


C:\Users\A1D\AppData\Local\Temp/ipykernel_14376/3540852770.py:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  image = torch.tensor(image)


batch of 50 images being processed...s
x1: torch.Size([5, 3750])  text:  torch.Size([5, 3164])
Batch discarded
batch of 50 images being processed...s
x1: torch.Size([5, 3750])  text:  torch.Size([5, 3164])
Batch discarded
batch of 50 images being processed...s
x1: torch.Size([5, 3750])  text:  torch.Size([5, 3164])
Batch discarded
batch of 50 images being processed...s
x1: torch.Size([5, 3750])  text:  torch.Size([5, 3164])
Batch discarded
batch of 50 images being processed...s
x1: torch.Size([5, 3750])  text:  torch.Size([5, 3164])
Batch discarded
batch of 50 images being processed...s
x1: torch.Size([5, 3750])  text:  torch.Size([5, 3164])
Batch discarded
batch of 50 images being processed...s
x1: torch.Size([5, 3750])  text:  torch.Size([5, 3164])
Batch discarded
batch of 50 images being processed...s
x1: torch.Size([5, 3750])  text:  torch.Size([5, 3164])
Batch discarded
batch of 50 images being processed...s
x1: torch.Size([5, 3750])  text:  torch.Size([5, 3164])
Batch discarded
b

Batch discarded
batch of 50 images being processed...s
x1: torch.Size([5, 3750])  text:  torch.Size([5, 3164])
Batch discarded
batch of 50 images being processed...s
x1: torch.Size([5, 3750])  text:  torch.Size([5, 3164])
Batch discarded
batch of 50 images being processed...s
x1: torch.Size([5, 3750])  text:  torch.Size([5, 3164])
Batch discarded
batch of 50 images being processed...s
x1: torch.Size([5, 3750])  text:  torch.Size([5, 3164])
Batch discarded
batch of 50 images being processed...s
x1: torch.Size([5, 3750])  text:  torch.Size([5, 3164])
Batch discarded
batch of 50 images being processed...s
x1: torch.Size([5, 3750])  text:  torch.Size([5, 3164])
Batch discarded
batch of 50 images being processed...s


In [ ]:
import pickle

with open('meme_model.pkl', 'wb') as f:
    pickle.dump(model, f)

    f.close()
